In [20]:
# SQL Alchemy
from sqlalchemy import create_engine

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# Config variables
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd
from config import local_db_user, local_db_pwd, local_db_endpoint, local_db_port, local_db_name

# Import Pandas
import pandas as pd

import psycopg2

# Extract CSVs into DataFrames

In [2]:
covid_cases_file = 'covid-cases-by-state.csv'
covid_cases = pd.read_csv(covid_cases_file, encoding= 'unicode_escape')
covid_cases.head()

,#,State,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,Total Cases per million,Total Deaths per million,Tests,Tests per million,Total Population
0,1.0,California,"20,653,265","35,919","356,743",298.0,"12,182,160","8,114,362","62,396","1,078","254,907,318","770,107","39,512,223"
1,2.0,Texas,"2,345,657",NaN,"26,294",NaN,"971,395","1,347,968","59,365",665,"33,058,311","836,660","28,995,881"
2,3.0,Florida,"1,784,569",NaN,"28,506",NaN,"1,453,639","302,424","61,546",983,"15,832,807","546,036","21,477,737"
3,4.0,New York,"1,323,315",NaN,"21,673",NaN,"735,381","566,261","61,613","1,009","15,703,599","731,157","19,453,561"
4,5.0,Illinois,"1,031,053",NaN,"38,145",NaN,"478,718","514,190","53,001","1,961","25,504,313","1,311,036","12,671,821"


In [3]:
relief_funds_file = 'HHS_Provider_Relief_Fund.csv'
relief_funds = pd.read_csv(relief_funds_file, encoding= 'unicode_escape')
relief_funds.head()

,Provider Name,State,City,Payment
0,BRANDON ASTIN DMD LLC,AK,ANCHOR POINT,"$19,003"
1,ELIZABETH WATNEY,AK,ANCHOR POINT,$724
2,A HAND UP BEHAVIOR SERVICES,AK,ANCHORAGE,"$1,191"
3,A JOINT EFFORT PHYSICAL THERAPY,AK,ANCHORAGE,"$11,641"
4,"AA PAIN CLINIC, INC.",AK,ANCHORAGE,"$69,976"


In [4]:
state_abbreviations_file = 'state_abbreviations.csv'
state_abbreviations = pd.read_csv(state_abbreviations_file, encoding='utf-8-sig')
state_abbreviations.head()


,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


# Transform DataFrames

In [5]:
#extract only relavent columns
covid_cases = covid_cases[['State', 'Total Cases', 'Total Deaths', 'Total Cases per million', 'Total Deaths per million', 'Tests', 'Tests per million', 'Total Population']]
covid_cases

,State,Total Cases,Total Deaths,Total Cases per million,Total Deaths per million,Tests,Tests per million,Total Population
0,California,"20,653,265","356,743","62,396","1,078","254,907,318","770,107","39,512,223"
1,Texas,"2,345,657","26,294","59,365",665,"33,058,311","836,660","28,995,881"
2,Florida,"1,784,569","28,506","61,546",983,"15,832,807","546,036","21,477,737"
3,New York,"1,323,315","21,673","61,613","1,009","15,703,599","731,157","19,453,561"
4,Illinois,"1,031,053","38,145","53,001","1,961","25,504,313","1,311,036","12,671,821"
...,...,...,...,...,...,...,...,...
60,Grand Princess Ship,"23,429",813,NaN,NaN,"204,307",NaN,NaN
61,Wuhan Repatriated,122,7,NaN,NaN,NaN,NaN,NaN
62,Diamond Princess Ship,3,NaN,NaN,NaN,3,NaN,NaN
63,Total:,46,NaN,NaN,NaN,46,NaN,NaN


In [6]:
relief_funds['Payment'] = relief_funds['Payment'].replace({'\$':''}, regex = True)
relief_funds['Payment'] = relief_funds['Payment'].replace({' ':''}, regex = True)
relief_funds['Payment'] = relief_funds['Payment'].replace({',':''}, regex = True)
relief_funds['Payment'] = relief_funds['Payment'].astype(float)
relief_funds.dtypes
#relief_funds

Provider Name     object
State             object
City              object
Payment          float64
dtype: object

In [7]:
payments_by_state = relief_funds[['State', 'Payment']].groupby('State').sum()
len(payments_by_state)
payments_by_state = payments_by_state.reset_index()
payments_by_state['Payment'] = payments_by_state['Payment'].map("${:,}".format)
payments_by_state.head()

,State,Payment
0,AK,"$222,894,177.0"
1,AL,"$1,304,513,045.0"
2,AR,"$925,881,569.0"
3,AS,"$5,950,917.0"
4,AZ,"$1,473,716,024.0"


# Create Local Database Connection

In [8]:
# Create Engine and Pass in MySQL Connection

# local Postgres DB Connection  
local_engine = create_engine(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}")
#print(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}")

# Create a local database engine connection
local_conn = local_engine.connect()

In [9]:
# Create local database
try:
    local_conn.execution_options(isolation_level="AUTOCOMMIT").execute(f"CREATE DATABASE {local_db_name}")
except Exception as e:
    print(e)

# Connect to local database
try:
    local_engine = create_engine(f"postgresql://{local_db_user}:{local_db_pwd}@{local_db_endpoint}:{local_db_port}/{local_db_name}")
    local_conn = local_engine.connect()
except Exception as e:
    print(e)  
    
#confirm tables
local_engine.table_names()

(psycopg2.errors.DuplicateDatabase) database "etl_project_db" already exists

[SQL: CREATE DATABASE etl_project_db]
(Background on this error at: http://sqlalche.me/e/13/f405)


['relief_provider',
 'relief_payments_by_state',
 'covid_cases',
 'state_abbreviations',
 'merged_df']

# Create Cloud Database Connection

In [25]:
# Create Engine and Pass in MySQL Connection

# Cloud MySQL Database Connection on AWS
cloud_engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}")

# Create a remote database engine connection
cloud_conn = cloud_engine.connect()

In [26]:
# Create cloud database
try:
    cloud_conn.execution_options(isolation_level="AUTOCOMMIT").execute(f"CREATE DATABASE {remote_db_name}")
except Exception as e:
    print(e)
    
# Connect to cloud database
try:
    cloud_engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
    cloud_conn = cloud_engine.connect()
except Exception as e:
    print(e)   
    
#confirm tables
cloud_engine.table_names()

(pymysql.err.ProgrammingError) (1007, "Can't create database 'etl_project_db'; database exists")
[SQL: CREATE DATABASE etl_project_db]
(Background on this error at: http://sqlalche.me/e/13/f405)


[]

# Load DataFrames into Local Database

In [13]:
#upload dataframe to database
relief_funds.to_sql(name='relief_provider', if_exists='replace', con=local_conn, index=False)
payments_by_state.to_sql(name='relief_payments_by_state', if_exists='replace', con=local_conn, index=False)
covid_cases.to_sql(name='covid_cases', if_exists='replace', con=local_conn, index=False)
state_abbreviations.to_sql(name='state_abbreviations', if_exists='replace', con=local_conn, index=False)

In [14]:
#confirm tables
local_engine.table_names()

['merged_df',
 'relief_provider',
 'relief_payments_by_state',
 'covid_cases',
 'state_abbreviations']

# Load DataFrames into Cloud Database

In [28]:
#upload dataframe to database
relief_funds.to_sql(name='relief_provider', if_exists='replace', con=cloud_conn, index=False)
payments_by_state.to_sql(name='relief_payments_by_state', if_exists='replace', con=cloud_conn, index=False)
covid_cases.to_sql(name='covid_cases', if_exists='replace', con=cloud_conn, index=False)
state_abbreviations.to_sql(name='state_abbreviations', if_exists='replace', con=cloud_conn, index=False)

In [29]:
#confirm tables
cloud_engine.table_names()

['covid_cases',
 'relief_payments_by_state',
 'relief_provider',
 'state_abbreviations']

# Create Merged Table and Load to Databases

In [17]:
#create merged table
merged_df = pd.read_sql('''
                            select 
                                state_abbreviations."Code" as "State Code",
                                state_abbreviations."State" as state,
                                covid_cases."Total Cases", 
                                covid_cases."Total Deaths",
                                relief_payments_by_state."Payment"

                            from
                                covid_cases 

                            inner join
                                state_abbreviations 

                            on 
                                covid_cases."State" = state_abbreviations."State"

                            inner join
                                relief_payments_by_state

                            on
                                relief_payments_by_state."State" = state_abbreviations."Code"
                        ''', 
                        local_engine)

merged_df.head()

,State Code,state,Total Cases,Total Deaths,Payment
0,AL,Alabama,"375,178","12,423","$1,304,513,045.0"
1,AK,Alaska,"58,064",930,"$222,894,177.0"
2,AZ,Arizona,"539,545","6,748","$1,473,716,024.0"
3,AR,Arkansas,"229,556","2,879","$925,881,569.0"
4,CA,California,"20,653,265","356,743","$8,436,326,765.0"


In [ ]:
pd.read_sql

In [18]:
#load merged_df into local dataframe
merged_df.to_sql(name='merged_df', if_exists='replace', con=local_conn, index=False)

#confirm tables
local_engine.table_names()

['relief_provider',
 'relief_payments_by_state',
 'covid_cases',
 'state_abbreviations',
 'merged_df']

In [30]:
#load merged_df into cloud dataframe
merged_df.to_sql(name='merged_df', if_exists='replace', con=cloud_conn, index=False)

#confirm tables
cloud_engine.table_names()

['covid_cases',
 'merged_df',
 'relief_payments_by_state',
 'relief_provider',
 'state_abbreviations']